## Startup Stuff (imports + lqr)

In [39]:
import numpy as np
import sympy as sym
from scipy import linalg
from scipy import signal

define the lqr function

In [40]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

## Exam Try 1


### Q1: L Place Poles
- transpose everything

In [5]:
A = np.array([[-0.30, 0.40, -0.70], [0.30, 0.00, -0.20], [-0.40, -0.70, -0.60]])
B = np.array([[0.30], [-0.30], [-0.90]])
C = np.array([[-0.20, -0.40, -0.90]])
p = np.array([-1.74+0.00j, -8.41-4.04j, -8.41+4.04j])

signal.place_poles(A.T, C.T, p).gain_matrix.T

array([[-5881.20858411],
       [ 4178.57833992],
       [ -569.83291016]])

### Q2: Observable?

In [6]:
A1 = np.array([[-2.5, 2.0, -9.8, -7.8], [-15.3, 11.3, -54.0, -42.7], [3.8, -2.6, 14.3, 11.7], [-7.7, 5.4, -28.1, -22.7]])
A2 = np.array([[-6.6, -10.9, -61.1], [-22.3, -36.5, -204.8], [4.7, 7.7, 43.2]])
A3 = np.array([[-33.3, -146.5, 388.3, -1553.2], [-0.2, -1.7, 3.2, -12.8], [-10.6, -43.5, 120.1, -480.4], [-1.9, -7.5, 21.2, -84.8]])
A4 = np.array([[-7.2, 28.3, -113.2], [-21.0, 90.3, -361.2], [-4.8, 20.8, -83.2]])
A5 = np.array([[-4.9, 22.0, 52.3, 88.0], [-39.4, 183.6, 439.5, 734.4], [5.1, -23.0, -55.8, -92.0], [6.5, -30.8, -73.3, -123.2]])
C1 = np.array([[0.5, -0.7, 3.9, 3.1]])
C2 = np.array([[0.1, 0.5, 2.5]])
C3 = np.array([[0.6, 2.6, -6.9, 27.6]])
C4 = np.array([[0.4, -1.7, 6.8]])
C5 = np.array([[-0.7, 3.0, 7.3, 12.0]])

wo1 = np.block([C1.T, A1.T@C1.T, A1.T@A1.T@C1.T, A1.T@A1.T@A1.T@C1.T]).T
wo2 = np.block([C2.T, A2.T@C2.T, A2.T@A2.T@C2.T]).T
wo3 = np.block([C3.T, A3.T@C3.T, A3.T@A3.T@C3.T, A3.T@A3.T@A3.T@C3.T]).T
wo4 = np.block([C4.T, A4.T@C4.T, A4.T@A4.T@C4.T]).T
wo5 = np.block([C5.T, A5.T@C5.T, A5.T@A5.T@C5.T, A5.T@A5.T@A5.T@C5.T]).T

In [7]:
np.linalg.inv(wo1)

array([[-3.54717145e-10, -1.17975588e+01, -1.59466193e+00,
         9.03151846e+01],
       [ 1.00000000e+01, -1.06078822e+02, -4.24312348e+01,
         6.05898045e+02],
       [ 1.00000000e+01, -4.09857509e+01,  6.71521690e+01,
         1.59907113e+02],
       [-1.00000000e+01,  2.95122686e+01, -9.38058040e+01,
        -7.89247422e+01]])

In [8]:
np.linalg.inv(wo2)

array([[    9.99999999,  -166.66666655, -1666.6666653 ],
       [   49.99999996,  -466.66666628, -5499.99999548],
       [   -9.99999999,    99.99999992,  1166.66666571]])

In [9]:
np.linalg.inv(wo3)

LinAlgError: Singular matrix

In [10]:
np.linalg.inv(wo4)

LinAlgError: Singular matrix

In [11]:
np.linalg.inv(wo5)

LinAlgError: Singular matrix

### Q3: Controller + Observer Design

In [13]:
w1, w2, w3, tau = sym.symbols('w1, w2, w3, tau')

j1 = 2.08
j2 = 2.33
j3 = 3.13

wd1 = (2+(j2-j3)*w2*w3)/j1
wd2 = ((j3-j1)*w3*w1)/j2
wd3 = (tau + (j1-j2)*w1*w2)/j3

f = sym.Matrix([wd1, wd2, wd3])
g = sym.Matrix([w2])

w3e = 3
w1e = 0
w2e = 2 / (-(j2-j3)*w3e)
taue = -(j1-j2)*w1e*w2e

A_num = sym.lambdify((w1, w2, w3, tau), f.jacobian([w1, w2, w3]))
B_num = sym.lambdify((w1, w2, w3, tau), f.jacobian([tau]))
C_num = sym.lambdify((w1, w2, w3, tau), g.jacobian([w1, w2, w3]))

In [14]:
w1e, w2e, w3e, taue

(0, 0.8333333333333335, 3, 0.0)

In [15]:
A = A_num(w1e, w2e, w3e, taue)
B = B_num(w1e, w2e, w3e, taue)
C = C_num(w1e, w2e, w3e, taue)

A, B, C

(array([[ 0.        , -1.15384615, -0.32051282],
        [ 1.35193133,  0.        ,  0.        ],
        [-0.06656017, -0.        ,  0.        ]]),
 array([[0.        ],
        [0.        ],
        [0.31948882]]),
 array([[0, 1, 0]]))

#### w observer

In [16]:
wobs = np.block([C.T, A.T@C.T, A.T@A.T@C.T]).T

In [17]:
wobs, np.linalg.matrix_rank(wobs)

(array([[ 0.        ,  1.        ,  0.        ],
        [ 1.35193133,  0.        ,  0.        ],
        [ 0.        , -1.55992077, -0.43331132]]),
 3)

#### LQR for L

In [18]:
Qo = np.array([[1.40]])
Ro = np.array([[2.10, 0.60, -0.30], [0.60, 2.30, 0.45], [-0.30, 0.45, 2.40]])

L = lqr(A.T, C.T, np.linalg.inv(Ro), np.linalg.inv(Qo)).T

In [19]:
L

array([[ 0.4423982 ],
       [ 1.37615614],
       [-0.85264303]])

#### W Controller

In [20]:
wcnt = np.block([B, A@B, A@A@B])

In [21]:
wcnt, np.linalg.matrix_rank(wcnt)

(array([[ 0.        , -0.10240026,  0.        ],
        [ 0.        ,  0.        , -0.13843812],
        [ 0.31948882,  0.        ,  0.00681578]]),
 3)

#### LQR for K

In [22]:
Qc = np.array([[2.70, 0.05, 0.35], [0.05, 3.90, 0.15], [0.35, 0.15, 2.60]])
Rc = np.array([[0.90]])

K = lqr(A, B, Qc, Rc)

In [23]:
K

array([[-1.486259  ,  2.35308382,  2.42300107]])

### Q4: A-LC roots

In [24]:
A = np.array([[0.4, -0.9], [-0.9, 0.9]])
B = np.array([[-0.9], [0.0]])
C = np.array([[0.0, 0.6]])
L1 = np.array([[6.6], [3.2]])
L2 = np.array([[-13.1], [0.5]])
L3 = np.array([[-53.4], [4.2]])
L4 = np.array([[-16.4], [-11.7]])
L5 = np.array([[-0.3], [1.7]])

w1 = A-L1@C
w2 = A-L2@C
w3 = A-L3@C
w4 = A-L4@C
w5 = A-L5@C

In [25]:
np.linalg.eigvals(w1)

array([ 1.89864212, -2.51864212])

In [26]:
np.linalg.eigvals(w2)

array([0.5+2.50079987j, 0.5-2.50079987j])

In [27]:
np.linalg.eigvals(w3)

array([-0.61+5.19672012j, -0.61-5.19672012j])

In [28]:
np.linalg.eigvals(w4)

array([1.69188331, 6.62811669])

In [29]:
np.linalg.eigvals(w5)

array([ 0.98593144, -0.70593144])

### Q5: xhat Controller Implementation
- X IS A ONE-D ARRAY FUCK THIS SHIT

In [ ]:
class Controller:
    def __init__(self):
        self.A = np.array([[0.0000, 1.0000], [0.0000, 0.0000]])
        self.B = np.array([[0.0000], [5.0930]])
        self.C = np.array([[1.0000, 0.0000]])
        
        self.K = np.array([[0.2538, 1.0728]])
        self.L = np.array([[0.6866], [0.2130]])
        
        self.dt = 0.01
        
        self.qe = np.pi/7
    
    def reset(self):
        self.xhat = np.array([0., 0.])
    
    def run(self, t, wheel_angle_measurement):
        """
        INPUTS
         t = current time (s)
         wheel_angle_measurement = measured angle of wheel (rad)
        
        OUTPUTS
         wheel_torque = torque applied to wheel by motor on ground (N m)
         """
        
        u = -self.K @ self.xhat
        
        y = np.array([wheel_angle_measurement - self.qe])
        
        self.xhat += (self.A@self.xhat + self.B@u - self.L@(self.C@self.xhat - y))*self.dt
        
        wheel_torque = u[0]
        print(wheel_torque)
        
        return wheel_torque

### Q6: Error = xhat - x

In [30]:
A = np.array([[-0.10, 0.20], [-0.60, -0.20]])
B = np.array([[0.90], [0.60]])
C = np.array([[0.10, 0.60]])
L = np.array([[-56.80], [24.30]])
x0 = np.array([-0.80, 0.90])
xhat0 = np.array([-1.50, 0.80])
t0 = 0.80
t1 = 1.00

f = A-L@C

x1 = linalg.expm(f*(t1-t0))@x0
xhat1 = linalg.expm(f*(t1-t0))@xhat0

In [32]:
xhat1 - x1

array([-1.1175188 ,  0.21013286])

### Q7: graphs q/r
- greater q/r ratio = tighter xhat vs. y line

In [33]:
one = 2e7/20000
two = 0.02 / 2
three = 30/3
four = 10000/10000

one, two, three, four

(1000.0, 0.01, 10.0, 1.0)

### Exam 1 Score

In [83]:
exam1_score = 70/75

In [84]:
exam1_score

0.9333333333333333

## Exam Try 2

### reset variables rq

In [37]:
%reset

### Q1: Observable?

In [43]:
A1 = np.array([[1.0, -2.0], [0.5, -1.0]])
A2 = np.array([[1.6, 2.5, -2.5], [0.6, 3.9, -3.9], [1.1, 4.5, -4.5]])
A3 = np.array([[-7.3, 36.5, -162.1, 73.0], [-46.4, 247.6, -1087.1, 495.2], [-27.2, 144.7, -635.3, 289.4], [-37.2, 197.3, -866.4, 394.6]])
A4 = np.array([[-240.5, -1155.0, 3238.9, 13196.1], [104.8, 503.7, -1410.9, -5748.4], [125.9, 605.0, -1696.6, -6912.3], [-26.1, -125.4, 351.8, 1433.3]])
A5 = np.array([[21.5, -113.3, -387.5, 983.5], [164.2, -864.4, -2948.8, 7473.2], [-106.7, 561.6, 1916.3, -4857.2], [-23.6, 124.2, 423.9, -1074.6]])
C1 = np.array([[-0.1, 0.2]])
C2 = np.array([[0.2, 0.5, -0.5]])
C3 = np.array([[0.8, -4.3, 18.9, -8.6]])
C4 = np.array([[-8.8, -42.3, 118.6, 483.3]])
C5 = np.array([[-2.0, 10.5, 35.8, -90.6]])

wo1 = np.block([C1.T]).T
wo2 = np.block([C2.T, A2.T@C2.T, A2.T@A2.T@C2.T]).T
wo3 = np.block([C3.T, A3.T@C3.T, A3.T@A3.T@C3.T, A3.T@A3.T@A3.T@C3.T]).T
wo4 = np.block([C4.T, A4.T@C4.T, A4.T@A4.T@C4.T, A4.T@A4.T@A4.T@C4.T]).T
wo5 = np.block([C5.T, A5.T@C5.T, A5.T@A5.T@C5.T, A5.T@A5.T@A5.T@C5.T]).T

In [44]:
np.linalg.inv(wo1)

LinAlgError: Last 2 dimensions of the array must be square

In [45]:
np.linalg.inv(wo2)

LinAlgError: Singular matrix

In [46]:
np.linalg.inv(wo3)

LinAlgError: Singular matrix

In [47]:
np.linalg.inv(wo4)

array([[ 9.99690411e+00, -5.20859741e+02,  1.40060985e+03,
        -6.10778339e+02],
       [ 1.08766575e-03,  7.09608917e+01, -2.16885010e+02,
         6.33896948e+02],
       [-3.99966347e+01, -2.07739426e+02, -5.48218447e+02,
         2.14878230e+03],
       [ 9.99921299e+00,  4.77053093e+01,  1.41050773e+02,
        -4.82943492e+02]])

In [48]:
np.linalg.inv(wo5)

array([[ 6.11505598e-04,  9.19171771e+02,  2.73336248e+03,
         2.41672087e+03],
       [-5.00010211e+01, -3.16676305e+02, -2.08338499e+03,
        -4.16675867e+03],
       [ 4.00006978e+01,  3.10839800e+02,  1.69170164e+03,
         2.83339605e+03],
       [ 1.00001439e+01,  6.58346588e+01,  3.66673858e+02,
         5.83346255e+02]])

### Q2: L Place Poles

In [52]:
A = np.array([[-0.90, 0.40], [-0.60, -0.60]])
B = np.array([[-0.50], [-0.90]])
C = np.array([[-0.50, -0.30]])
p = np.array([-5.07-3.28j, -5.07+3.28j])

signal.place_poles(A.T, C.T, p).gain_matrix.T

array([[ 33.38788945],
       [-84.44648241]])

### Q3: A-LC

In [53]:
A = np.array([[0.6, -0.5, -0.2], [0.7, -0.3, -0.8], [0.6, -0.4, 0.4]])
B = np.array([[0.2], [0.3], [0.2]])
C = np.array([[-0.2, 0.5, -0.9]])
L1 = np.array([[-164.1], [-39.8], [12.1]])
L2 = np.array([[97.1], [-875.1], [-514.4]])
L3 = np.array([[-173.5], [142.4], [127.6]])
L4 = np.array([[-324.0], [324.5], [265.0]])
L5 = np.array([[-194.0], [-314.7], [-120.1]])

f1 = A-L1@C
f2 = A-L2@C
f3 = A-L3@C
f4 = A-L4@C
f5 = A-L5@C

In [54]:
np.linalg.eigvals(f1)

array([-0.21468451+5.29690547j, -0.21468451-5.29690547j,
       -0.90063098+0.j        ])

In [55]:
np.linalg.eigvals(f2)

array([-3.5952609 +8.10079271j, -3.5952609 -8.10079271j,
        1.90052181+0.j        ])

In [56]:
np.linalg.eigvals(f3)

array([ 6.36253546,  4.97541143, -1.69794689])

In [57]:
np.linalg.eigvals(f4)

array([ 8.43736406,  5.90980751, -2.19717157])

In [58]:
np.linalg.eigvals(f5)

array([5.48014868+4.92404002j, 5.48014868-4.92404002j,
       0.19970264+0.j        ])

### Q4: error = xhat - x

In [59]:
A = np.array([[-0.40, 0.20, 0.10], [0.20, -0.70, 0.80], [-0.60, -0.50, -0.20]])
B = np.array([[-0.70], [0.00], [-0.70]])
C = np.array([[0.70, 0.60, -0.40]])
L = np.array([[-1009.90], [1440.30], [366.10]])
x0 = np.array([-1.00, 1.00, -0.50])
xhat0 = np.array([-0.80, 0.90, -1.40])
t0 = 0.40
t1 = 0.90

f = A-L@C
x1 = linalg.expm(f*(t1-t0))@x0
xhat1 = linalg.expm(f*(t1-t0))@xhat0

error = xhat1-x1

In [60]:
error

array([-5.3943384 ,  6.77239925,  0.87286442])

### Q5: Observer and Controller Design

In [62]:
w1, w2, w3, tau = sym.symbols('w1, w2, w3, tau')

j1 = 3.73
j2 = 3.38
j3 = 6.13

wd1 = (tau + (j2-j3)*w2*w3)/j1
wd2 = ((j3-j1)*w3*w1)/j2
wd3 = (2+(j1-j2)*w1*w2)/j3

f = sym.Matrix([wd1, wd2, wd3])
g = sym.Matrix([w2])

w1e = 2
w2e = 2 / (-(j1-j2)*w1e)
w3e = 0
taue = -(j2-j3)*w2e*w3e

A_num = sym.lambdify((w1, w2, w3, tau), f.jacobian([w1, w2, w3]))
B_num = sym.lambdify((w1, w2, w3, tau), f.jacobian([tau]))
C_num = sym.lambdify((w1, w2, w3, tau), g.jacobian([w1, w2, w3]))

equilibrium values

In [63]:
w1e, w2e, w3e, taue

(2, -2.8571428571428563, 0, -0.0)

A, B, C values

In [74]:
A = A_num(w1e, w2e, w3e, taue).astype(float)
B = B_num(w1e, w2e, w3e, taue).astype(float)
C = C_num(w1e, w2e, w3e, taue).astype(float)

A, B, C

(array([[ 0.        , -0.        ,  2.10647262],
        [ 0.        ,  0.        ,  1.42011834],
        [-0.16313214,  0.1141925 ,  0.        ]]),
 array([[0.26809651],
        [0.        ],
        [0.        ]]),
 array([[0., 1., 0.]]))

w observer

In [75]:
wobs = np.block([C.T, A.T@C.T, A.T@A.T@C.T]).T

In [76]:
wobs, np.linalg.matrix_rank(wobs)

(array([[ 0.        ,  1.        ,  0.        ],
        [ 0.        ,  0.        ,  1.42011834],
        [-0.23166694,  0.16216686,  0.        ]]),
 3)

LQR for L

In [77]:
Qo = np.array([[0.80]])
Ro = np.array([[2.20, 0.25, 0.40], [0.25, 2.40, -0.05], [0.40, -0.05, 3.20]])

L = lqr(A.T, C.T, np.linalg.inv(Ro), np.linalg.inv(Qo)).T

In [78]:
L

array([[0.98621741],
       [1.4154206 ],
       [0.58645614]])

w controller

In [68]:
wcnt = np.block([B, A@B, A@A@B])

In [69]:
wcnt, np.linalg.matrix_rank(wcnt)

(array([[ 0.26809651,  0.        , -0.09212691],
        [ 0.        ,  0.        , -0.0621091 ],
        [ 0.        , -0.04373516,  0.        ]]),
 3)

lqr for K

In [70]:
Qc = np.array([[2.40, 0.05, -0.55], [0.05, 3.00, 0.35], [-0.55, 0.35, 2.60]])
Rc = np.array([[0.50]])

K = lqr(A, B, Qc, Rc)

In [71]:
K

array([[  4.63006131,  -6.15514656, -13.67127044]])

### Q6: Xhat Controller Implementation

In [ ]:
class Controller:
    def __init__(self):
        self.A = np.array([[0.0000, 1.0000], [0.0000, 0.0000]])
        self.B = np.array([[0.0000], [5.0930]])
        self.C = np.array([[1.0000, 0.0000]])

        self.K = np.array([[0.6255, 1.5209]])
        self.L = np.array([[2.5998], [2.2574]])

        self.dt = 0.01
        self.qe = np.pi/8
    
    def reset(self):
        self.xhat = ([0.,0.])
    
    def run(self, t, wheel_angle_measurement):
        """
        INPUTS
         t = current time (s)
         wheel_angle_measurement = measured angle of wheel (rad)
        
        OUTPUTS
         wheel_torque = torque applied to wheel by motor on ground (N m)
        """
        
        u = -self.K@self.xhat
        y = np.array([wheel_angle_measurement - self.qe])
        self.xhat += (self.A@self.xhat + self.B@u - self.L@(self.C@self.xhat - y))*self.dt

        wheel_torque = u[0]
        
        return wheel_torque

### Q7: q/r ratio graphs

In [79]:
one = 3/3
two = 0.3/30
three = 2000/2
four = 200000/20000

one, two, three, four

(1.0, 0.01, 1000.0, 10.0)

### Exam Try 2 Score

In [80]:
exam2_score = 74.92/75

In [81]:
exam2_score

0.9989333333333333

## Scores!

Exam Try 1

In [85]:
exam1_score

0.9333333333333333

Exam Try 2

In [86]:
exam2_score

0.9989333333333333